#### This notebook handles spaceship titanic dataset in which i drop all columns except for three.
###### The outcome is a decrease in accuracy [0.74]
---
So much for finetuning :laughing:

In [1]:
import numpy as np
from sklearn import *
import xgboost as xgb

In [2]:
# Importing the data
import pandas as pd
titanic_test = pd.read_csv(r"C:\Users\user\Desktop\ML_Marathon\Data\Datasets\spaceship-titanic\test.csv")
titanic_train = pd.read_csv(r"C:\Users\user\Desktop\ML_Marathon\Data\Datasets\spaceship-titanic\train.csv")
titanic_train.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported'],
      dtype='object')

In [3]:
titanic_train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [7]:
len(titanic_train.Spa.unique())

1328

In [9]:
#Storing PassengerId column to a variable
PassId = titanic_test.PassengerId

# Dropping columns
titanic_test = titanic_test.drop(['PassengerId','Cabin','Name','VRDeck','Spa','ShoppingMall','FoodCourt','RoomService'],axis=1)
titanic_train = titanic_train.drop(['PassengerId','Cabin','Name','VRDeck','Spa','ShoppingMall','FoodCourt','RoomService'],axis=1)

In [11]:
# Handling Missing values
titanic_train = titanic_train.fillna({'HomePlanet': titanic_train['HomePlanet'].bfill(), 'Destination': titanic_train['Destination'].bfill(),'Age':titanic_train.Age.mean(),'VIP':False})
titanic_test = titanic_test.fillna({'HomePlanet': titanic_test['HomePlanet'].bfill(), 'Destination': titanic_test['Destination'].bfill(),'Age':titanic_test.Age.mean(),'VIP':False})
titanic_train.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,Transported
0,Europa,False,TRAPPIST-1e,39.0,False,False
1,Earth,False,TRAPPIST-1e,24.0,False,True
2,Europa,False,TRAPPIST-1e,58.0,True,False
3,Europa,False,TRAPPIST-1e,33.0,False,False
4,Earth,False,TRAPPIST-1e,16.0,False,True


In [12]:
titanic_train = titanic_train.fillna({'CryoSleep':titanic_train.CryoSleep.bfill()})

In [14]:
#Encoding the values
lb = preprocessing.LabelBinarizer()
le = preprocessing.LabelEncoder()
#lb.fit(titanic_train.VIP) # true or false values
dest_tran = preprocessing.LabelEncoder()
planet_tran = preprocessing.LabelEncoder()
cryo_tran = preprocessing.LabelEncoder()
dest_tran = preprocessing.LabelEncoder()
# Train file
titanic_train['HomePlanet'] = planet_tran.fit_transform(titanic_train['HomePlanet'])
titanic_train['Destination']  = dest_tran.fit_transform(titanic_train['Destination'])
titanic_train['CryoSleep'] = cryo_tran.fit_transform(titanic_train['CryoSleep'])
titanic_train['VIP'] = lb.fit_transform(titanic_train['VIP'])
# Test file
titanic_test['CryoSleep'] = cryo_tran.fit_transform(titanic_test['CryoSleep'])
titanic_test['Destination']  = dest_tran.fit_transform(titanic_test['Destination'])
titanic_test['HomePlanet'] = planet_tran.fit_transform(titanic_test['HomePlanet'])
titanic_test['VIP'] = lb.fit_transform(titanic_test['VIP'])

# Encoded Labels
print(dest_tran.classes_,planet_tran.classes_,lb.classes_,cryo_tran.classes_)

['55 Cancri e' 'PSO J318.5-22' 'TRAPPIST-1e'] ['Earth' 'Europa' 'Mars'] [False  True] [False True nan]


In [19]:
titanic_train.iloc[:,:-1].head()

,HomePlanet,CryoSleep,Destination,Age,VIP
0,1,0,2,39.0,0
1,0,0,2,24.0,0
2,1,0,2,58.0,1
3,1,0,2,33.0,0
4,0,0,2,16.0,0


In [50]:
# Trainining the data
X_train = titanic_train.iloc[:,:-1]
y_train = titanic_train.iloc[:,-1]
X_test = titanic_test

# Models
sk_model = tree.DecisionTreeClassifier()
xgb_model = xgb.XGBClassifier()

sk_model.fit(X_train,y_train)
sk_predict = sk_model.predict(X_test)

xgb_model.fit(X_train,y_train)
xgb_predict = xgb_model.predict(X_test)

# Fine tuning
params = {'max_depth':[None,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]}
sk_grid_model = model_selection.GridSearchCV(sk_model,params,cv=20)
xgb_grid_model = model_selection.GridSearchCV(xgb_model,params,cv=20)

sk_grid_model.fit(X_train,y_train)
sk_grid_predict = sk_grid_model.predict(X_test)

xgb_grid_model.fit(X_train,y_train)
xgb_grid_predict = xgb_grid_model.predict(X_test)

# Differences in Accuracy
acc = metrics.accuracy_score(xgb_grid_predict,sk_grid_predict)
print(acc)

0.9801262567220014


In [51]:
xgb_grid_model.best_params_

{'max_depth': 2}

In [26]:
iris = datasets.load_iris()
iris.target_names
iris.feature_names[2:]

['petal length (cm)', 'petal width (cm)']

In [29]:
xgb_grid_model.best_estimator_

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=2, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [27]:
feature_names = titanic_train.columns[:-1]
class_names = titanic_train.Transported.unique()

In [34]:
# Decision Tree Image
tree.export_graphviz(
 sk_model,
 out_file="xgb_grid_model_titanic_tree.dot",
 feature_names=feature_names,
 class_names=class_names,
 rounded=True,
 filled=True
 )


TypeError: can only concatenate str (not "numpy.bool_") to str

In [31]:
tree.export_graphviz??

Signature:
tree.export_graphviz(
    decision_tree,
    out_file=None,
    *,
    max_depth=None,
    feature_names=None,
    class_names=None,
    label='all',
    filled=False,
    leaves_parallel=False,
    impurity=True,
    node_ids=False,
    proportion=False,
    rotate=False,
    rounded=False,
    special_characters=False,
    precision=3,
    fontname='helvetica',
)
Source:   
@validate_params(
    {
        "decision_tree": "no_validation",
        "out_file": [str, None, HasMethods("write")],
        "max_depth": [Interval(Integral, 0, None, closed="left"), None],
        "feature_names": ["array-like", None],
        "class_names": ["array-like", "boolean", None],
        "label": [StrOptions({"all", "root", "none"})],
        "filled": ["boolean"],
        "leaves_parallel": ["boolean"],
        "impurity": ["boolean"],
        "node_ids": ["boolean"],
        "proportion": ["boolean"],
        "rotate": ["boolean"],
        "rounded": ["boolean"],
        "special_charac

In [ ]:
# Testing the data

In [35]:
fine_tune2 = pd.Series(sk_grid_predict,index=PassId)

In [36]:
fine_tune2.head()

PassengerId
0013_01     True
0018_01    False
0019_01     True
0021_01    False
0023_01    False
dtype: bool

In [56]:
data = {
    'PassengerId':PassId,
    'Transported':sk_grid_predict
}
fine_tune2 = pd.DataFrame(data,index=None)

In [55]:
fine_tune2.to_csv('fine_tune5_xgb.csv',index=None)

In [42]:
x = pd.read_csv(r'C:\Users\user\Desktop\ML_Marathon\EDA\predictions\fi_ne_tune.csv')

In [43]:
x.head()

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True


In [44]:
acc1 = metrics.accuracy_score(x.Transported,xgb_grid_predict)

In [45]:
acc1

0.801028758475567

In [46]:
fine_tune2 = pd.read_csv(r'C:\Users\user\Desktop\ML_Marathon\EDA\fine_tune2.csv')

In [47]:
metrics.accuracy_score(x.Transported,fine_tune2.Transported)

0.8078092120645313